# 0

In [1]:
!pip install transformers

     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.6 kB ? eta -:--:--
     ----------------- -------------------- 20.5/43.6 kB 162.5 kB/s eta 0:00:01
     -------------------------- ----------- 30.7/43.6 kB 145.2 kB/s eta 0:00:01
     ----------------------------------- -- 41.0/43.6 kB 178.6 kB/s eta 0:00:01
     -------------------------------------- 43.6/43.6 kB 163.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.3 MB 991.0 kB/s eta 0:00:10
   ---------------------------------------- 0.1/9.3 MB 825.8 kB/s eta 0:00:12
   ---------------------------------------- 0.1/9.3 MB 581.0 kB/s eta 0:00:16
    --------------------------------------- 0.2/9.3 MB 876.1 kB/s eta 0:00:11
    --------------------------------------- 0.2/9.3 MB 985.7 kB/s eta 0:00:10
   - -------------------------------------- 0.2/9.3 MB 885.4 kB/s eta 0:00:1


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pythonw.exe -m pip install --upgrade pip


In [1]:
!pip install torch scikit-learn scipy tensorboard tensorflow tf


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pythonw.exe -m pip install --upgrade pip


In [2]:
!pip install tensorflow tf


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pythonw.exe -m pip install --upgrade pip


In [3]:
%pip install -e fast_transformers tensorflow_datasets

Obtaining file:///C:/Users/HP/Documents/hard/D.I/CSC%201401/OnLearningTheKernelmain/fast_transformers
Note: you may need to restart the kernel to use updated packages.


ERROR: file:///C:/Users/HP/Documents/hard/D.I/CSC%201401/OnLearningTheKernelmain/fast_transformers does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%cd OnLearningTheKernelmain 
!python setup.py build_ext

[WinError 2] The system cannot find the file specified: 'OnLearningTheKernelmain'
c:\Users\HP\Documents\hard\D.I\CSC 1401\OnLearningTheKernelmain


c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


running build_ext
building 'fast_transformers.hashing.hash_cpu' extension
ninja: no work to do.
"C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.40.33807\bin\HostX86\x64\link.exe" /nologo /INCREMENTAL:NO /LTCG /DLL /MANIFEST:EMBED,ID=2 /MANIFESTUAC:NO "/LIBPATH:c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\Lib\site-packages\torch\lib" "/LIBPATH:c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\libs" /LIBPATH:c:\Users\HP\AppData\Local\Programs\Python\Python311\libs /LIBPATH:c:\Users\HP\AppData\Local\Programs\Python\Python311 "/LIBPATH:c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\PCbuild\amd64" "/LIBPATH:C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.40.33807\lib\x64" "/LIBPATH:C:\Program Files (x86)\Windows Kits\10\lib\10.0.22621.0\ucrt\x64" "/LIBPATH:C:\Program Files (x86)\Windows Kits\10\\lib\10.0.22621.0\\um\x64" c10.lib torch.lib torch_cpu.lib torch_python.lib /EXPORT:PyInit_hash_cpu "c:\Users\HP\Documents\hard\D.I\CSC 14

c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\Lib\site-packages\torch\utils\cpp_extension.py:384: UserWarning: Error checking compiler version for cl: [WinError 2] The system cannot find the file specified
  warnings.warn(f'Error checking compiler version for {compiler}: {error}')
c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\Lib\site-packages\torch\utils\cpp_extension.py:418: UserWarning: The detected CUDA version (12.5) has a minor version mismatch with the version that was used to compile PyTorch (12.1). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
Emitting ninja build file c:\Users\HP\Documents\hard\D.I\CSC 1401\OnLearningTheKernelmain\build\temp.win-amd64-cpython-311\Release\build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Emitting ninja build file c:\Users\HP\Documents\hard\D.I\CSC 1401\OnLearningTheKerne

In [2]:
import torch
from fast_transformers.builders import AttentionBuilder 
# import fast_transformers

In [ ]:
best_loss = 1 

In [10]:
import os 
import math 
import torch 
from sys import argv
from time import time
import numpy as np 
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForCausalLM

from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.tensorboard import SummaryWriter

from text.data import IMBDbByteDataset
from text.model import ByteTextClassifier
from tqdm import tqdm

LOG_PATH  = './log/IMDB-4K/mem/'

EPOCHS = 6
LOG_FREQ = 100
BATCH_SIZE = 16

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

def predict(model, data_loader, device): 
    # Turn on the evaluation mode
    model.eval() 

    # Initialize outputs
    labels = np.array([])
    predictions = np.array([])
     
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader): 
            # Unpack batch 
            y = batch['label'].to(device)
            x = batch['tokens'].to(device)
            length_mask = batch['length'].to(device)

            # Call model 
            logits = model(x, length_mask)
            probs = nn.functional.softmax(logits, dim=1)
            y_hat = torch.argmax(probs, dim=1)

            labels = np.concatenate((labels, \
                                    y.detach().cpu().numpy()))

            predictions = np.concatenate((predictions, \
                                         y_hat.detach().cpu().numpy()))

    return labels, predictions

valid_dataset = IMBDbByteDataset(
    training_set=False, 
    sequence_len=20
)

valid_loader = DataLoader(
    valid_dataset, 
    BATCH_SIZE, 
    shuffle=False, 
    drop_last=False
)

train_dataset = IMBDbByteDataset(
    training_set=True, 
    sequence_len=20
)


# Specify model parameters
params = {      
    'n_classes': 2, 
    'classifier_dim': 1024, 
    'max_len': 20, 
    'd_model': 256, 
    'attention_type': 'rbf-fastfood', 
    'n_layers': 4, 
    'n_heads' : 4,
    'd_ff': 1024, 
    'd_query': 64, 
    'd_values': 64,
    'activation': 'gelu', 
    'output_norm': True
}

# Model 
model = ByteTextClassifier(
    vocab_size=train_dataset.vocab_size, 
    **params
)
model = torch.nn.DataParallel(model)



# Set precision 
model.float()
state = torch.load(os.path.join(LOG_PATH, 'final.pt'))
model.load_state_dict(state)
model.to(device)
print("Start predicting")
y_valid, y_valid_pred = predict(
    model, 
    valid_loader, 
    device
)
valid_accuracy = 100*accuracy_score(y_valid, y_valid_pred)
print(valid_accuracy)

cuda:0
Start predicting
50.0


In [3]:
import os 
import math 
import torch 
from sys import argv
from time import time
import numpy as np 
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForCausalLM

from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.tensorboard import SummaryWriter

from text.data import IMBDbByteDataset
from text.model import ByteTextClassifier
from tqdm import tqdm

# model = AutoModelForCausalLM.from_pretrained("model")

if len(argv)==2:
    argv.append('4000')
LOG_PATH  = './log/IMDB-4K/mem/'

EPOCHS = 6
LOG_FREQ = 100
BATCH_SIZE = 16
WARMUP_STEPS = 8000
LEARNING_RATE = 5e-2

def get_learning_rate_scheduler(optimizer, num_warmup_steps, last_epoch=-1): 
    """
    Create learning rate scheduler. 

    Arguments:
    ----------
        optimizer (:class:`~torch.optim.Optimizer`):
            The optimizer for which to schedule the learning rate.
        num_warmup_steps (:obj:`int`):
            The number of steps for the warmup phase.
        last_epoch (:obj:`int`, `optional`, defaults to -1):
            The index of the last epoch when resuming training.

    Returns:
    --------
        :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
    """
    def lr_lambda(current_step: int): 
        r = 1.0 

        # Warmup phase
        r *= min(1.0, float(current_step) / float(num_warmup_steps))

        # Decay phase
        r /= math.sqrt(max(current_step, num_warmup_steps))

        return r

    return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch)

def evaluate(model, data_loader, criterion, device): 
    # Turn on the evaluation mode
    model.eval() 

    total_loss = 0.0 

    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader): 
            # Unpack batch 
            y = batch['label'].to(device).long()
            x = batch['tokens'].to(device)
            length_mask = batch['length'].to(device)

            # Call model 
            logits = model(x, length_mask)

            # Accumulate loss 
            total_loss += len(y) * criterion(logits, y).item()

    return total_loss / len(data_loader.dataset)

def predict(model, data_loader, device): 
    # Turn on the evaluation mode
    model.eval() 

    # Initialize outputs
    labels = np.array([])
    predictions = np.array([])
     
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader): 
            # Unpack batch 
            y = batch['label'].to(device)
            x = batch['tokens'].to(device)
            length_mask = batch['length'].to(device)

            # Call model 
            logits = model(x, length_mask)
            probs = nn.functional.softmax(logits, dim=1)
            y_hat = torch.argmax(probs, dim=1)

            labels = np.concatenate((labels, \
                                    y.detach().cpu().numpy()))

            predictions = np.concatenate((predictions, \
                                         y_hat.detach().cpu().numpy()))

    return labels, predictions

if __name__ == '__main__':
    # Specify GPU option 
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(device)

    # Specify model parameters
    params = {
        'n_classes': 2, 
        'classifier_dim': 1024, 
        'max_len': 1400, 
        'd_model': 256, 
        'attention_type': 'fsgb-fastfood', 
        'n_layers': 4, 
        'n_heads' : 4,
        'd_ff': 1024, 
        'd_query': 64, 
        'd_values': 64,
        'activation': 'gelu', 
        'output_norm': True
    }

    # Load datasets 
    train_dataset = IMBDbByteDataset(
        training_set=True, 
        sequence_len=params['max_len']
    )

    valid_dataset = IMBDbByteDataset(
        training_set=False, 
        sequence_len=params['max_len']
    )

    # Dataloaders 
    train_loader = DataLoader(
        train_dataset, 
        BATCH_SIZE, 
        shuffle=True, 
        drop_last=False
    )
    valid_loader = DataLoader(
        valid_dataset, 
        BATCH_SIZE, 
        shuffle=False, 
        drop_last=False
    )

    # Model 
    model = ByteTextClassifier(
        vocab_size=train_dataset.vocab_size, 
        **params
    )
    model = torch.nn.DataParallel(model)

    # Set precision 
    model.float()
    state = torch.load(os.path.join(LOG_PATH, 'final.pt'))
    if state:
        
        model.load_state_dict(state)
    # Send to GPU 
    model.to(device)

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Optimizer 
    optimizer = optim.AdamW(
        model.parameters(), 
        lr=LEARNING_RATE, 
        betas=(0.9, 0.98),
        eps=1e-9, 
        weight_decay=1e-1
    )

    # Scheduler 
    scheduler = get_learning_rate_scheduler(
        optimizer, 
        WARMUP_STEPS
    )

    # Tensorboard
    writer = SummaryWriter(LOG_PATH)
    best_valid_loss = float('inf')    
    best_valid_acc = 0
    # Repeat for each epoch 
    for epoch in tqdm(range(EPOCHS)): 
        # Turn on the training mode
        model.train()

        # Loop over batches 
        for train_batch_idx, train_batch in enumerate(train_loader):
            st=time()
            # Unpack batch 
            y = train_batch['label'].to(device).long()
            x = train_batch['tokens'].to(device)
            length_mask = train_batch['length'].to(device)

            # Reset graph 
            optimizer.zero_grad()

            # Call model 
            logits = model(x, length_mask)
            #print(" calulating loss")
            # Loss 
            loss = criterion(logits, y)
            print(loss);
            loss.backward()
            #print("Finish calulating loss")
            # Optimization step 
            optimizer.step()
            #print("optimizer step")

            # Update learning rate 
            scheduler.step()
            et=time()
            #print(et-st)
            # Report results
            if train_batch_idx>0 and train_batch_idx % LOG_FREQ == 0:  
                print(4)
                # Learning rate 
                writer.add_scalar(
                    'Learning Rate', 
                    scheduler.get_last_lr()[0],
                    scheduler._step_count
                )

                # Training loss
                writer.add_scalar(
                    'Training Loss', 
                    loss.item(), 
                    scheduler._step_count
                )

                print(5)
                # Training accuracy 
                y_pred = torch.argmax(
                    nn.functional.softmax(logits, dim=1), 
                    dim=1
                )
                print(5)

                writer.add_scalar(
                    'Training Accuracy', 
                    100*accuracy_score(
                        y.detach().cpu().numpy(), 
                        y_pred.detach().cpu().numpy()
                    ), 
                    scheduler._step_count
                )
                # print(6)
                # # Validation loss
                # valid_loss = evaluate(
                #     model, 
                #     valid_loader, 
                #     criterion, 
                #     device
                # )
                # print(7)
                # writer.add_scalar(
                #     'Validation Loss', 
                #     valid_loss,
                #     scheduler._step_count
                # )
                # print(8)

                # # Validation accuracy 
                # y_valid, y_valid_pred = predict(
                #     model, 
                #     valid_loader, 
                #     device
                # )
                # print(9)
                # writer.add_scalar(
                #     'Validation Accuracy', 
                #     100*accuracy_score(y_valid, y_valid_pred), 
                #     scheduler._step_count
                # )
                # print(10)
                # valid_accuracy=100*accuracy_score(y_valid, y_valid_pred)
                # print(valid_accuracy,100*accuracy_score(
                #         y.detach().cpu().numpy(), 
                #         y_pred.detach().cpu().numpy()
                #     ) )
                # print(11)
                # if valid_accuracy > best_valid_acc:
                #     best_valid_acc = valid_accuracy
                #     # Save new model 
                #     torch.save(
                #         model.state_dict(), 
                #         os.path.join(LOG_PATH, 'accuracy-best.pt')
                #     )
                # print(12)
                # # Reset training mode 
                model.train()
                if(loss < best_loss):
                    best_loss = loss
                    torch.save(
                        model.state_dict(), 
                        os.path.join('model.pt')
                    )
        # Evaluate the model
        print(13) 
        valid_loss = evaluate(
            model, 
            valid_loader, 
            criterion, 
            device
        )
        print(14) 
        y_valid, y_valid_pred = predict(
            model, 
            valid_loader, 
            device
        )
        print(15)
        valid_accuracy = 100*accuracy_score(y_valid, y_valid_pred)

        print(16)
        # Save best validation loss model 
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            # Save new model 
            torch.save(
                model.state_dict(), 
                os.path.join(LOG_PATH, 'loss-best.pt')
            )
        print(17)
        # Save best validation accuracy model 
        if valid_accuracy > best_valid_acc:
            best_valid_acc = valid_accuracy
            # Save new model 
            torch.save(
                model.state_dict(), 
                os.path.join(LOG_PATH, 'accuracy-best.pt')
            )
    print(18)
    writer.close()

    print(19)
    # Save final model 
    torch.save(
        model.state_dict(), 
        os.path.join(LOG_PATH, 'final.pt')
    )
    print(20)
    memory_stats = torch.cuda.memory_stats()
    print(21)
    max_memory_reserved = torch.cuda.max_memory_reserved(device=device)
    print(22)
    max_memory_allocated = torch.cuda.max_memory_allocated(device=device)

    print('=' * 45, ' Final Checkpoint ', '=' * 45)

    # Training set 
    train_loss = evaluate(model, train_loader, criterion, device)
    y_train, y_train_pred = predict(model, train_loader, device)
    C_train, C_train_counts = np.unique(y_train, return_counts=True)

    # Validation set 
    valid_loss = evaluate(model, valid_loader, criterion, device)
    y_valid, y_valid_pred = predict(model, valid_loader, device)
    C_valid, C_valid_counts = np.unique(y_valid, return_counts=True)

    print('| Training Loss {:5.4f} | Training Accuracy {:8.2f} | F1 Score {:8.2f}'.format(\
        train_loss, 100*accuracy_score(y_train, y_train_pred), 
        100*f1_score(y_train, y_train_pred, average='micro')))

    print('| Validation Loss {:5.4f} | Validation Accuracy {:8.2f} | F1 Score {:8.2f}'.format(\
        valid_loss, 100*accuracy_score(y_valid, y_valid_pred), 
        100*f1_score(y_valid, y_valid_pred, average='micro')))

    print('=' * 45, ' Loss Checkpoint ', '=' * 45)

    # Load checkpoint
    model.load_state_dict(torch.load(os.path.join(LOG_PATH, 'loss-best.pt')))

    # Training set 
    train_loss = evaluate(model, train_loader, criterion, device)
    y_train, y_train_pred = predict(model, train_loader, device)
    C_train, C_train_counts = np.unique(y_train, return_counts=True)

    # Validation set 
    valid_loss = evaluate(model, valid_loader, criterion, device)
    y_valid, y_valid_pred = predict(model, valid_loader, device)
    C_valid, C_valid_counts = np.unique(y_valid, return_counts=True)

    print('| Training Loss {:5.4f} | Training Accuracy {:8.2f} | F1 Score {:8.2f}'.format(\
        train_loss, 100*accuracy_score(y_train, y_train_pred), 
        100*f1_score(y_train, y_train_pred, average='micro')))

    print('| Validation Loss {:5.4f} | Validation Accuracy {:8.2f} | F1 Score {:8.2f}'.format(\
        valid_loss, 100*accuracy_score(y_valid, y_valid_pred), 
        100*f1_score(y_valid, y_valid_pred, average='micro')))

    print('=' * 45, ' Accuracy Checkpoint ', '=' * 45)

    # Load checkpoint
    model.load_state_dict(torch.load(os.path.join(LOG_PATH, 'accuracy-best.pt')))

    # Training set 
    train_loss = evaluate(model, train_loader, criterion, device)
    y_train, y_train_pred = predict(model, train_loader, device)
    C_train, C_train_counts = np.unique(y_train, return_counts=True)

    # Validation set 
    valid_loss = evaluate(model, valid_loader, criterion, device)
    y_valid, y_valid_pred = predict(model, valid_loader, device)
    C_valid, C_valid_counts = np.unique(y_valid, return_counts=True)

    print('| Training Loss {:5.4f} | Training Accuracy {:8.2f} | F1 Score {:8.2f}'.format(\
        train_loss, 100*accuracy_score(y_train, y_train_pred), 
        100*f1_score(y_train, y_train_pred, average='micro')))

    print('| Validation Loss {:5.4f} | Validation Accuracy {:8.2f} | F1 Score {:8.2f}'.format(\
        valid_loss, 100*accuracy_score(y_valid, y_valid_pred), 
        100*f1_score(y_valid, y_valid_pred, average='micro')))

    print('=' * 45, ' Memory Statistics ', '=' * 45)

    print('Max Memory Reserved: ', max_memory_reserved)
    print('Max Memory Allocated: ', max_memory_allocated)

    peak_bytes_requirement = memory_stats["allocated_bytes.all.peak"]

    print('Peak Allocated Bytes: ', peak_bytes_requirement)


cuda:0


  0%|          | 0/6 [00:00<?, ?it/s]

tensor(0.6872, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6866, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7016, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6823, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7009, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7305, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6829, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7092, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7031, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6866, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6656, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7019, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6869, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6810, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6872, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6697, device='cuda:0', grad_fn=

 17%|█▋        | 1/6 [1:38:21<8:11:45, 5901.09s/it]

15
16
17
tensor(0.7033, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6868, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7080, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6721, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6989, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7020, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7031, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6768, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6838, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6730, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6659, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7080, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6415, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6501, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7018, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6835, device='cuda:0',

 33%|███▎      | 2/6 [6:24:01<13:54:10, 12512.54s/it]

15
16
17
tensor(0.6959, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7117, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7807, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7591, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7480, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8251, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6974, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6538, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7096, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6819, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6697, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6864, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6893, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6796, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6954, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6762, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6787, device='cuda:0',

 33%|███▎      | 2/6 [7:35:53<15:11:46, 13676.65s/it]


KeyboardInterrupt: 

In [23]:
from gettext import install


%cd hadamard_cuda  
!python setup.py install

%cd diag_mult_cuda 
!python setup.py install

[WinError 2] The system cannot find the file specified: 'hadamard_cuda'
c:\Users\HP\Documents\hard\D.I\CSC 1401\OnLearningTheKernelmain
running install
running bdist_egg
running egg_info
writing pytorch_fast_transformers.egg-info\PKG-INFO
writing dependency_links to pytorch_fast_transformers.egg-info\dependency_links.txt
writing requirements to pytorch_fast_transformers.egg-info\requires.txt
writing top-level names to pytorch_fast_transformers.egg-info\top_level.txt
reading manifest file 'pytorch_fast_transformers.egg-info\SOURCES.txt'
writing manifest file 'pytorch_fast_transformers.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
running build_ext
building 'fast_transformers.hashing.hash_cpu' extension
ninja: no work to do.
"C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.40.33807\bin\HostX86\x64\link.exe" /nologo /INCREMENTAL:NO /LTCG /DLL /MANIFEST:EMBED,ID=2 /MANIFESTUAC:NO "/LIBPATH:c:\

c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\Lib\site-packages\setuptools\_distutils\cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\Lib\site-packages\setuptools\_distutils\cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards

running install
running bdist_egg
running egg_info
writing pytorch_fast_transformers.egg-info\PKG-INFO
writing dependency_links to pytorch_fast_transformers.egg-info\dependency_links.txt
writing requirements to pytorch_fast_transformers.egg-info\requires.txt
writing top-level names to pytorch_fast_transformers.egg-info\top_level.txt
reading manifest file 'pytorch_fast_transformers.egg-info\SOURCES.txt'
writing manifest file 'pytorch_fast_transformers.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
running build_ext
building 'fast_transformers.hashing.hash_cpu' extension
ninja: no work to do.
"C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.40.33807\bin\HostX86\x64\link.exe" /nologo /INCREMENTAL:NO /LTCG /DLL /MANIFEST:EMBED,ID=2 /MANIFESTUAC:NO "/LIBPATH:c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\Lib\site-packages\torch\lib" "/LIBPATH:c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\libs" /L

c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\Lib\site-packages\setuptools\_distutils\cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
c:\Users\HP\Documents\hard\D.I\CSC 1401\.venv\Lib\site-packages\setuptools\_distutils\cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards